# DOMIN🁏CAML

_Projet tutoré de DUT 2 informatique (Aristide GRANGE, Université de Lorraine, 2021)_

## Préambule

La cellule ci-dessous importe un fichier `"tests.ml"` qui reprend dans l'ordre alphabétique l'intégralité des exemples de ce document, sauf les deux derniers. Il importe lui-même le fichier `"main.ml"`, dans lequel vous devrez écrire votre propre programme.

In [1]:
#use "tests.ml"

type domino = D of int * int
type chain = E | S of int * string * int
type player = H of int | B of int
val read : (unit -> string) ref = {contents = <fun>}
val input_valid : string -> (string -> bool) -> (string -> 'a) -> 'a = <fun>
val flip : domino -> domino = <fun>
val append : domino * chain * char -> chain = <fun>
val legal_adds : domino -> chain -> chain list = <fun>
val possible_dominoes : domino list -> chain -> domino list = <fun>
val suppress : domino -> domino list -> domino list = <fun>
val input_move :
  (domino list -> domino) ->
  (chain -> chain -> chain) ->
  chain -> domino list -> (domino list * chain) option = <fun>
val input_bot_move : chain -> domino list -> (domino list * chain) option =
  <fun>
val input_human_move : chain -> domino list -> (domino list * chain) option =
  <fun>
val take : domino list -> int -> domino list -> domino list * domino list =
  <fun>
val string_of_player : player -> string = <fun>
val get_hand_size : int -> int = <fun>
val char_list_

Si vous avez installé OCaml, vous pouvez également lancer les tests en ligne de commande ainsi:
```console
ocaml tests.ml
```

Matériel de jeu
---------------

Il consiste en 28 **dominos** formés de toutes les paires possibles de 0
à $n$ points, $n$ étant traditionnellement 6.
Un domino a deux **moitiés** (p. ex. 1 et 2 pour le domino
2-1). Les dominos sont **symétriques** lorsque leurs deux moitiés sont
égales (p. ex. 6-6, le double-six). Un domino possède deux
**présentations** distinctes (p. ex. 1-2 et 2-1) ou non (p. ex. 6-6 et
6-6), dites **permutées**.

Déroulement d'une partie
------------------------

Deux, trois ou quatre joueurs s'affrontent, chacun d'eux pouvant être
soit un **humain**, soit un **bot**. Chacun des deux \[resp.
trois, quatre\] joueurs reçoit initialement une **main** (_hand_) de 6 \[resp.
7\] dominos. Le reste des dominos constitue le **talon** (_stack_).

![*Une chaîne de dominos compatibles.*](https://www.dropbox.com/s/zu9s65wu6jto2l3/compatibles.png?raw=1)

Le premier joueur pose le domino de son choix. Ensuite, à tour de rôle,
chaque joueur pose un domino **compatible** à l'un des deux **bouts** de
la **chaîne** des dominos déjà posés. S'il n'a aucun domino à poser, il
en **pioche** deux (si possible) et **passe son tour**.

Le premier à avoir posé tous ses dominos gagne. Il y a match nul lorsque
la pioche est vide et que tout le monde doit passer son tour.

Idée de l'implantation
----------------------

### Types construits

Les dominos seront simplement représentés par un type produit formé de deux entiers.

```ocaml
type domino = D of int * int
```

Voici 🁏 et 🀱. Notez que l'ordre des membres du couple, ou **présentation** n'importe pas.

In [2]:
[D (2, 4), D (0, 0)]

- : (domino * domino) list = [(D (2, 4), D (0, 0))]


Le talon et les différentes mains seront gérés avec des listes comme ci-dessus.

La chaîne sera un type somme représentant:
- soit la chaîne vide initiale;
- soit le produit du chiffre de l'extrémité gauche (_west_), d'une chaîne de caractères prête à afficher, et enfin du chiffre de l'extrémité droite (_east).

```ocaml
type chain = E | S of int * string * int
```

Ainsi, la chaîne de dominos 🁌🁠🂋🁔🁣🀵 s'écrira:

In [3]:
S (3, "3-6 6-5 5-5 5-0 0-0 0-4", 4)

- : chain = S (3, "3-6 6-5 5-5 5-0 0-0 0-4", 4)


Enfin, un joueur sera représenté par sa catégorie (soit `H` pour humain, soit `B` pour bot) suivie de son numéro.

```ocaml
type player = H of int | B of int
```

Voici le troisième joueur:

In [4]:
B 3

- : player = B 3


### Mode opératoire

Le ou les bots joueront au hasard, dans la limite des coups **légaux**.

Le gros du travail consistera justement à déterminer à tout moment
l'ensemble de ces coups, ce qui permettra également de détecter les
situations suivantes:

-   le joueur a le choix entre plusieurs coups, et il faut lui demander
    d'exprimer ce choix;

-   le joueur n'a qu'un seul coup jouable, et il faut le jouer pour lui;

-   le joueur doit passer son tour, il faut le faire piocher;

-   le jeu est bloqué, il faut terminer sur un match nul.

## Gestion des coups légaux

### `flip`
La fonction
```ocaml
val flip : domino -> domino = <fun>
```
permute un domino donné.

In [5]:
flip (D (2,3))

- : domino = D (3, 2)


In [6]:
flip (D (3,3))

- : domino = D (3, 3)


### `append`
La fonction
```ocaml
val append : domino * chain * char -> chain = <fun>
```
ajoute un domino donné à une extrémité donnée d'une chaîne donnée, sans effectuer aucun test de compatibilité sur la chaîne, le domino ou sa présentation.

In [7]:
append (D (5, 1), S (1, "1-2 4-3", 3), '<')

- : chain = S (5, "5-1 1-2 4-3", 3)


In [8]:
append (D (3, 5), S (1, "1-2 4-3", 3), '>')

- : chain = S (1, "1-2 4-3 3-5", 5)


In [9]:
append (D (5, 2), S (1, "1-2 4-3", 3), '<')

- : chain = S (5, "5-2 1-2 4-3", 3)


### `legal_adds`
La fonction
```ocaml
val legal_adds : domino -> chain -> chain list = <fun>
```
renvoie les chaînes de dominos résultant de toutes les poses légales et **distinctes** d'un domino
`d` donné au bout d'une chaîne de dominos `cd` donnée. Référez-vous au fichier `dominos_tests.ml`
fourni pour le descriptif de chacun des cas illustrés ci-dessous (cet avis est général et ne sera
pas répété pour chaque fonction).

In [10]:
legal_adds (D (5, 6)) E

- : chain list = [S (5, "5-6", 6)]


In [11]:
legal_adds (D (5, 6)) (S (1, "1-5 6-2", 2))

- : chain list = []


In [12]:
legal_adds (D (1, 5)) (S (5, "5-5 5-3 3-2", 2))

- : chain list = [S (1, "1-5 5-5 5-3 3-2", 2)]


In [13]:
legal_adds (D (5, 1)) (S (5, "5-5 5-3 3-2", 2))

- : chain list = [S (1, "1-5 5-5 5-3 3-2", 2)]


In [14]:
legal_adds (D (2, 4)) (S (5, "5-5 5-3 3-2", 2))

- : chain list = [S (5, "5-5 5-3 3-2 2-4", 4)]


In [15]:
legal_adds (D (4, 2)) (S (5, "5-5 5-3 3-2", 2))

- : chain list = [S (5, "5-5 5-3 3-2 2-4", 4)]


In [16]:
legal_adds (D (5, 6)) (S (6, "6-1 1-5", 5))

- : chain list = [S (5, "5-6 6-1 1-5", 5); S (6, "6-1 1-5 5-6", 6)]


In [17]:
legal_adds (D (6, 5)) (S (6, "6-1 1-5", 5))

- : chain list = [S (5, "5-6 6-1 1-5", 5); S (6, "6-1 1-5 5-6", 6)]


In [18]:
legal_adds (D (6, 6)) (S (6, "6-0 0-1", 1))

- : chain list = [S (6, "6-6 6-0 0-1", 1)]


In [19]:
legal_adds (D (6, 6)) (S (1, "1-0 0-6", 6))

- : chain list = [S (1, "1-0 0-6 6-6", 6)]


In [20]:
legal_adds (D (5, 6)) (S (6, "6-1 1-6", 6))

- : chain list = [S (5, "5-6 6-1 1-6", 6)]


In [21]:
legal_adds (D (6, 6)) (S (6, "6-1 1-6", 6))

- : chain list = [S (6, "6-6 6-1 1-6", 6)]


L'évaluation de votre solution tiendra largement compte de sa concision
et de sa clarté: ne vous jetez pas tête baissée dans la programmation,
et réfléchissez soigneusement à la manière de regrouper les cas!

### `possible_dominoes`
La fonction
```ocaml
val possible_dominoes : domino list -> chain -> domino list = <fun>
```
renvoie la liste de chacun des dominos d'une main donnée qui est plaçable au bout d'une chaîne donnée.

In [22]:
possible_dominoes [D (3,2); D (5,4); D (3,3); D (6,3)] (S (3, "3-4 4-2 2-1 0-1", 1)) 

- : domino list = [D (3, 2); D (3, 3); D (6, 3)]


In [23]:
possible_dominoes [D (3,2); D (5,4); D (3,3); D (6,3)] (S (2, "2-4 4-3 3-1 0-1", 6)) 

- : domino list = [D (3, 2); D (6, 3)]


In [24]:
possible_dominoes [D (3,3); D (5,4); D (3,3); D (5,3)] (S (2, "2-4 4-3 3-1 0-1", 6)) 

- : domino list = []


## Sélection du coup à jouer et calcul du résultat
Un coup peut être joué soit par un humain, soit par un bot.

Si c'est un humain, il faut éventuellement lui demander ce qu'il veut jouer : quel domino parmi ceux qui sont possibles, et éventuellement à quel bout de chaîne le placer. Le résultat est un couple formé de la nouvelle main et de la nouvelle chaîne.

Il peut arriver que le joueur n'ait pas le choix: si un seul coup est possible, celui-ci est forcé; si aucun coup n'est possible, `None` est renvoyé pour déléguer la pioche à la fonction appelante.

Pour les coups joués par un bot, la même logique s'applique, à cette différence que les choix sont tirés au hasard plutôt que demandés.

### `input_valid`

#### Utilisation interactive avec la commande `ocaml`

Pour lire sur l'entrée standard, la fonction normale Ocaml est `read_line`. Elle marche avec la commande `ocaml` si vous l'avez intallée (pour lancer le REPL : `ocaml`, et pour lancer le programme : `ocaml main.ml`). La fonction de saisie jusqu'à validité est alors définie par le code ci-dessous, qui vous est gracieusement fourni: copiez-collez-le au début de votre fichier `"main.ml"`.

```ocaml
let read = ref read_line;
let rec input_valid prompt is_valid cast =
  let rec urs () =
    let s = !read () in
    if is_valid s then cast s
    else
      let () = print_endline "Réessayez!" in
      urs ()
  in
  print_endline prompt; urs ()
```

Cette fonction complètement générique prend en entrée un message d'invite, une fonction testant la validité de l'entrée (selon un critère quelconque) et une autre appliquant une conversion quelconque à une entrée valide avant de la renvoyer.

#### Utilisation interactive sous Jupyter Notebook

Cependant, sous Jupyter Notebook ou [TryOcamlPro](http://try.ocamlpro.com), `read_line` lève une erreur. J'ai ouvert une [issue GitHub](https://github.com/akabe/ocaml-jupyter/issues/162), et l'auteur du kernel Ocaml pour Jupyter Notebook  a eu la gentillesse de me proposer un contournement avec la fonction `Jupyter_comm.Stdin.read_line`. Pour ce faire, commencez par charger les bibliothèques nécessaires (des messages sur fond rouge devraient apparaître la première fois):

In [25]:
#use "topfind";;
#require "jupyter.comm";;

- : unit = ()
Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads

- : unit = ()


/Users/aristide/.opam/default/lib/result: added to search path
/Users/aristide/.opam/default/lib/result/result.cma: loaded
/Users/aristide/.opam/default/lib/ppx_deriving/runtime: added to search path
/Users/aristide/.opam/default/lib/ppx_deriving/runtime/ppx_deriving_runtime.cma: loaded
/Users/aristide/.opam/default/lib/ppx_deriving_yojson/runtime: added to search path
/Users/aristide/.opam/default/lib/ppx_deriving_yojson/runtime/ppx_deriving_yojson_runtime.cma: loaded
/Users/aristide/.opam/default/lib/ocaml/unix.cma: loaded
/Users/aristide/.opam/default/lib/bytes: added to search path
/Users/aristide/.opam/default/lib/uuidm: added to search path
/Users/aristide/.opam/default/lib/uuidm/uuidm.cma: loaded
/Users/aristide/.opam/default/lib/easy-format: added to search path
/Users/aristide/.opam/default/lib/easy-format/easy_format.cma: loaded
/Users/aristide/.opam/default/lib/biniou: added to search path
/Users/aristide/.opam/default/lib/biniou/biniou.cma: loaded
/Users/aristide/.opam/defa

Dans le bout de code que je vous ai donné plus haut, je suis subrepticement sorti du paradigme fonctionnel en définissant un alias **mutable** de la fonction normale `read_line` grâce au mot-clé `ref`. On peut alors surcharger cette fonction en changeant sa valeur (avec l'opérateur d'affectation `:=`) :

In [26]:
read := (function () -> Jupyter_comm.Stdin.read_line "")

- : unit = ()


Voici par exemple la saisie interactive jusqu'à validité d'un âge, avec rendu de celui-ci augmenté de 20 %:

In [27]:
input_valid
    "Votre âge?"
    (function x -> let n = int_of_string x in n >= 0 && n < 120)
    (function x -> int_of_float (1.2 *. float_of_string x))

Votre âge?
-10
Réessayez!
500
Réessayez!
10


- : int = 12


#### Utilisation non interactive lors des tests

Quand vous lancez les tests fournis, vous n'avez certainement pas envie de saisir les mêmes entrées à chaque fois. Nous enregistrerons donc les chaînes désirées dans un flux, qui les délivrera ensuite une par une, simulant ainsi les entrées successives de l'utilisateur. Cela se fait toujours en changeant la valeur de `read`, cette fois de la manière suivante:

In [28]:
read := (let x = Stream.of_list ["-10"; "500"; "10"] in function () -> Stream.next x);

input_valid
    "Votre âge?"
    (function x -> let n = int_of_string x in n >= 0 && n < 120)
    (function x -> 1.2 *. float_of_string x |> int_of_float)

Votre âge?
Réessayez!
Réessayez!


- : int = 12


Les âge « saisis » sont les mêmes, mais ils n'apparaissent plus dans l'affichage, et c'est normal.

Notez que j'en ai profité pour écrire une variante du code précédent avec l'opérateur `|>`, utilisé ici pour présenter l'expression `int_of_float (1.2 *. float_of_string x)` dans un ordre plus naturel. N'hésitez pas à imiter ce style lorsqu'il peut clarifier une composition de fonctions.

### `suppress`

La fonction
```ocaml
val suppress : domino -> domino list -> domino list = <fun>
```
est utilisée pour rendre une copie mise à jour d'une main donnée lorsqu'on en retire un domino donné.

In [29]:
suppress (D (2,3)) []

- : domino list = []


In [30]:
suppress (D (2,3)) [D (1, 2); D (6, 5)]

- : domino list = [D (1, 2); D (6, 5)]


In [31]:
suppress (D (4,5)) [D (4,5)]

- : domino list = []


In [32]:
suppress (D (4,5)) [D (5,4)]

- : domino list = []


In [33]:
suppress (D (4,5)) [D (4,5); D (3,2)]

- : domino list = [D (3, 2)]


In [34]:
suppress (D (4,5)) [D (5,4); D (3,2)]

- : domino list = [D (3, 2)]


In [35]:
suppress (D (4,5)) [D (3,2); D (4,5)]

- : domino list = [D (3, 2)]


In [36]:
suppress (D (4,5)) [D (3,2); D (5,4)]

- : domino list = [D (3, 2)]


In [37]:
suppress (D (4,5)) [D (3,2); D (5,4); D (6, 3)]

- : domino list = [D (3, 2); D (6, 3)]


In [38]:
suppress (D (5,4)) [D (3,2); D (5,4); D (6, 3)]

- : domino list = [D (3, 2); D (6, 3)]


### `input_move`

La fonction
```ocaml
val input_move :
  (domino list -> domino) ->
  (chain -> chain -> chain) ->
  chain -> domino list -> (domino list * chain) option = <fun>
```
est une abstraction de l'acquisition d'un coup, prévue pour être utilisée aussi bien pour un joueur humain que pour un bot. La spécialisation se fait grâce aux deux premiers arguments.

Le premier argument est une fonction que j'appelle `select_domino` et qui, étant donné la main d'un joueur, renvoie le domino possible sélectionné par celui-ci (soit par saisie, soit par tirage). Elle ne sera pas appelée par `move` lorsqu'aucun domino de la main n'est possible, ou qu'un seul domino est possible.

Le deuxième argument est une fonction que j'appelle `select_end` et qui, étant donné deux chaînes de domino (résultant en fait de l'ajout du domino sélectionné à l'une ou l'autre extrémité de la chaîne en cours), en renvoie une (soit par saisie d'une direction `'>'` ou `'<'`, soit par tirage aléatoire).

Le troisième argument est la chaîne en cours.

Le quatrième argument est la main du joueur.

Le résultat est un couple formé de la nouvelle main et de la nouvelle chaîne si un coup est possible, ou rien sinon. Le talon n'intervenant pas à ce niveau, dans ce dernier cas la pioche sera déléguée à la fonction appelante.

Dans les exemples ci-dessous, pour simplifier, les deux fonctions de sélection passées en paramètre sont totalement déterministes. Notez l'affichage par effet de bord quand le coup est forcé.

In [39]:
input_move
    (function l -> failwith "should not occur")
    (fun dc1 dc2 -> failwith "should not occur")
    (S (6, "6-1 1-4", 4))
    [D (1, 2); D (3, 5); D (5, 3)]

- : (domino list * chain) option = None


In [40]:
input_move
    (function l -> List.nth l 0)
    (fun dc1 dc2 -> failwith "should not occur")
    (S (6, "6-1 1-4", 4))
    [D (1, 2); D (6, 5); D (5, 3)]

	coup forcé :         6-5


- : (domino list * chain) option =
Some ([D (1, 2); D (5, 3)], S (5, "5-6 6-1 1-4", 4))


In [41]:
input_move
    (function l -> List.nth l 0)
    (fun _ dc2 -> dc2)
    (S (6, "6-1 1-4", 4))
    [D (1, 2); D (6, 4); D (5, 3)]

	coup forcé :         6-4


- : (domino list * chain) option =
Some ([D (1, 2); D (5, 3)], S (6, "6-1 1-4 4-6", 6))


In [42]:
input_move
    (function l -> List.nth l 0)
    (fun dc1 _ -> dc1)
    (S (6, "6-1 1-4", 4))
    [D (1, 2); D (6, 4); D (5, 3)]

	coup forcé :         6-4


- : (domino list * chain) option =
Some ([D (1, 2); D (5, 3)], S (4, "4-6 6-1 1-4", 4))


In [43]:
input_move
    (function l -> List.nth l 0)
    (fun dc1 dc2 -> failwith "should not occur")
    (S (6, "6-1 1-4", 4))
    [D (1, 2); D (6, 5); D (4, 3)]

- : (domino list * chain) option =
Some ([D (1, 2); D (4, 3)], S (5, "5-6 6-1 1-4", 4))


In [44]:
input_move
    (function l -> List.nth l 1)
    (fun dc1 dc2 -> failwith "should not occur")
    (S (6, "6-1 1-4", 4))
    [D (1, 2); D (6, 5); D (4, 3)]

- : (domino list * chain) option =
Some ([D (1, 2); D (6, 5)], S (6, "6-1 1-4 4-3", 3))


In [45]:
input_move
    (function l -> List.nth l 1)
    (fun dc1 _ -> dc1)
    (S (6, "6-1 1-4", 4))
    [D (1, 2); D (6, 5); D (6, 4)]

- : (domino list * chain) option =
Some ([D (1, 2); D (6, 5)], S (4, "4-6 6-1 1-4", 4))


In [46]:
input_move
    (function l -> List.nth l 1)
    (fun _ dc2 -> dc2)
    (S (6, "6-1 1-4", 4))
    [D (1, 2); D (6, 5); D (6, 4)]

- : (domino list * chain) option =
Some ([D (1, 2); D (6, 5)], S (6, "6-1 1-4 4-6", 6))


### `input_bot_move`

La fonction
```ocaml
val input_bot_move : chain -> domino list -> (domino list * chain) option = <fun>
```
est la spécialisation « bot » de `input_move`, avec la même sémantique pour les deux derniers arguments et le résultat. Pour la tester de façon déterministe, nous avons besoin de fixer au préalable le germe du générateur pseudo-aléatoire:

In [47]:
Random.init 42

- : unit = ()


Notez dans les évaluations ci-dessous que `input_bot_move` ajoute une couche d'affichage à `input_move` en affichant le cas échéant le domino à placer de façon à faciliter le suivi du jeu par un lecteur humain.

In [48]:
input_bot_move (S (6, "6-1 1-5", 5)) [D (6, 5)]

	coup forcé :         6-5


- : (domino list * chain) option = Some ([], S (5, "5-6 6-1 1-5", 5))


In [49]:
input_bot_move (S (6, "6-1 1-4", 4)) [D (6, 5)]

	coup forcé :         6-5


- : (domino list * chain) option = Some ([], S (5, "5-6 6-1 1-4", 4))


In [50]:
input_bot_move (S (6, "6-1 1-4", 4)) [D (6, 3); D (4, 2)]

	à placer :           4-2


- : (domino list * chain) option = Some ([D (6, 3)], S (6, "6-1 1-4 4-2", 2))


In [51]:
input_bot_move (S (6, "6-1 1-5", 5)) [D (4, 4)]

- : (domino list * chain) option = None


### `input_human_move`

La fonction
```ocaml
val input_human_move : chain -> domino list -> (domino list * chain) option = <fun>
```
est la spécialisation « human » de `input_move`, avec la même sémantique pour les deux derniers arguments et le résultat. Pour la tester de façon déterministe, nous avons besoin de fixer au préalable la suite des saisies du joueur:

In [52]:
read := (let x = Stream.of_list ["356"; "12"; "64"; "foobar"; ">"] in function () -> Stream.next x);

input_human_move
    (S (6, "6-1 1-4", 4))
    [D (1, 2); D (6, 5); D (6, 4)]

	Quel domino voulez-vous poser ? 
Réessayez!
Réessayez!
	À quel bout ? 
Réessayez!


- : (domino list * chain) option =
Some ([D (1, 2); D (6, 5)], S (6, "6-1 1-4 4-6", 6))


## Gestion complète d'un coup, avec affichages et pioche éventuelle

### `string_of_player`
La fonction
```ocaml
val string_of_player : player -> string = <fun>
```
calcule simplement la chaîne de caractères à afficher pour désigner un joueur.

In [53]:
string_of_player (H 2)

- : string = "Joueur 2 (humain)"


In [54]:
string_of_player (B 2)

- : string = "Joueur 2 (bot)   "


### `string_of_dominoes`

La fonction
```ocaml
val string_of_dominoes : domino list -> string = <fun>
```
calcule simplement la chaîne de caractères représentant une main de dominos.

In [55]:
string_of_dominoes []

- : string = ""


In [56]:
string_of_dominoes [D (5, 6)]

- : string = "5-6"


In [57]:
string_of_dominoes [D (5, 6); D (4, 4); D (3, 3); D (1, 1); D (2, 2)]

- : string = "5-6 4-4 3-3 1-1 2-2"


### `take`
La fonction
```ocaml
val take : domino list -> int -> domino list -> domino list * domino list = <fun>
```
transfère un nombre donné de dominos d'une liste donnée (talon) à une autre (main).

In [58]:
take [D (1, 1); D (2, 2)] 0 [D (3, 3); D (4, 4); D (5, 6)]

- : domino list * domino list =
([D (1, 1); D (2, 2)], [D (3, 3); D (4, 4); D (5, 6)])


In [59]:
take [D (1, 1); D (2, 2)] 2 [D (3, 3); D (4, 4); D (5, 6)]

- : domino list * domino list =
([D (4, 4); D (3, 3); D (1, 1); D (2, 2)], [D (5, 6)])


In [60]:
take [D (1, 1); D (2, 2)] 3 [D (3, 3); D (4, 4); D (5, 6)]

- : domino list * domino list =
([D (5, 6); D (4, 4); D (3, 3); D (1, 1); D (2, 2)], [])


In [61]:
take [D (1, 1); D (2, 2)] 42 [D (3, 3); D (4, 4); D (5, 6)]

- : domino list * domino list =
([D (5, 6); D (4, 4); D (3, 3); D (1, 1); D (2, 2)], [])


### `move`
La fonction
```ocaml
val move :
  domino list ->
  chain ->
  domino list ->
  player ->
  domino list * chain * domino list = <fun>
```
prend un talon, une chaîne, une main et un joueur, et renvoie le talon, la chaîne et la main résultant d'un nouveau coup, non sans produire en effet de bord tous les affichages associés. Nous la testons ci-dessous sans perte de généralité sur un coup joué par un bot.

In [62]:
Random.init 42

- : unit = ()


In [63]:
move [D (1, 1); D (1, 2); D (1, 3)] (S (6, "6-1 1-5", 5)) [D (6, 5)] (B 1)

Joueur 1 (bot)   :
	main : 6-5
	coup forcé :         6-5


- : domino list * chain * domino list =
([D (1, 1); D (1, 2); D (1, 3)], S (5, "5-6 6-1 1-5", 5), [])


In [64]:
move [D (1, 1); D (1, 2); D (1, 3)] (S (6, "6-1 1-4", 4)) [D (6, 5)] (B 2)

Joueur 2 (bot)   :
	main : 6-5
	coup forcé :         6-5


- : domino list * chain * domino list =
([D (1, 1); D (1, 2); D (1, 3)], S (5, "5-6 6-1 1-4", 4), [])


In [65]:
move [D (1, 1); D (1, 2); D (1, 3)] (S (6, "6-1 1-4", 4)) [D (6, 3); D (4, 2)] (B 3)

Joueur 3 (bot)   :
	main : 6-3 4-2
	à placer :           4-2


- : domino list * chain * domino list =
([D (1, 1); D (1, 2); D (1, 3)], S (6, "6-1 1-4 4-2", 2), [D (6, 3)])


In [66]:
move [D (1, 1); D (1, 2); D (1, 3)] (S (6, "6-1 1-5", 5)) [D (4, 4)] (B 4)

Joueur 4 (bot)   :
	main : 4-4
	Aucun coup possible!


- : domino list * chain * domino list =
([D (1, 3)], S (6, "6-1 1-5", 5), [D (1, 2); D (1, 1); D (4, 4)])


## Mise en place d'une partie

### `make_dominoes`
La fonction
```ocaml
val make_dominoes : int -> domino list = <fun>
```
génère sous forme de liste l'ensemble des dominos de plus haut chiffre donné. Par exemple, pour une liste de plus haut domino `D (3, 3)`:

In [67]:
make_dominoes 3

- : domino list =
[D (3, 3); D (3, 2); D (3, 1); D (3, 0); D (2, 2); D (2, 1); D (2, 0);
 D (1, 1); D (1, 0); D (0, 0)]


### `char_list_of_string`
La fonction
```ocaml
val char_list_of_string : string -> char list = <fun>
```
renvoie la liste des caractères d'une chaîne de caractères donnée.

In [68]:
char_list_of_string ""

- : char list = []


In [69]:
char_list_of_string "foobar"

- : char list = ['f'; 'o'; 'o'; 'b'; 'a'; 'r']


### `players_of_string`
```ocaml
val players_of_string : string -> player list = <fun>
```
convertit une chaîne formée de caractères `B` et `H` en une chaîne de valeurs de type `player` numérotées séquentiellement.

In [70]:
players_of_string "HBB"

- : player list = [H 1; B 2; B 3]


### `get_hand_size`
La fonction
```ocaml
val get_hand_size : int -> int = <fun>
```
renvoie le nombre de dominos dans la main initiale d'un joueur en fonction du nombre de ceux-ci. Elle doit échouer avec le message `"Entre 2 et 4 joueurs, please!"` pour tout autre entier que 2, 3 et 4.

In [71]:
get_hand_size 2

- : int = 7


In [72]:
get_hand_size 3

- : int = 6


In [73]:
get_hand_size 4

- : int = 6


### `make_state_list`
La fonction
```ocaml
val make_state_list : string -> domino list -> domino list * (domino list * player) list = <fun>
```
prend une chaîne formée de caractères B et H, et la liste des dominos du jeu (résultat de `make_dominoes`), et construit le talon initial, ainsi que la liste des **états** initiaux, un état étant le couple formé par un joueur et sa main. On voit dans l'exemple ci-dessous que pour un jeu de 15 dominos, les 7 premiers sont distribués au joueur 1, les 7 suivants au joueur 2, et qu'il ne reste plus que `D (0, 0)` dans le talon.

In [74]:
make_state_list
    "BH"
    [D (4, 4); D (4, 3); D (4, 2); D (4, 1); D (4, 0); D (3, 3); D (3, 2); D (3, 1); D (3, 0); D (2, 2); D (2, 1); D (2, 0); D (1, 1); D (1, 0); D (0, 0)]

- : domino list * (domino list * player) list =
([D (0, 0)],
 [([D (3, 2); D (3, 3); D (4, 0); D (4, 1); D (4, 2); D (4, 3); D (4, 4)],
   B 1);
  ([D (1, 0); D (1, 1); D (2, 0); D (2, 1); D (2, 2); D (3, 0); D (3, 1)],
   H 2)])


## Jeu proprement dit

### `string_of_chain`
La fonction
```ocaml
val string_of_chain : chain -> string = <fun>
```
extrait simplement le deuxième membre du triplet d'une chaîne de dominos donnée, ou à défaut renvoie la chaîne de caractères vide.

In [75]:
string_of_chain E

- : string = ""


In [76]:
string_of_chain (S (4, "4-5 5-6 6-6 6-2", 2))

- : string = "4-5 5-6 6-6 6-2"


### `string_of_state`
La fonction
```ocaml
val string_of_state : domino list * player -> string = <fun>
```
calcule la chaîne de caractères représentant l'état d'un joueur.

In [77]:
string_of_state ([D (1, 2); D (6, 5); D (4, 3)], H 1)

- : string = "Joueur 1 (humain):\t1-2 6-5 4-3"


### `list_shuffle`
La fonction
```ocaml
val list_shuffle : 'a list -> 'a list = <fun>
```
renvoie une copie mélangée d'une liste donnée. C'est l'exercice 10.4 de votre fascicule, auquel je vous renvoie pour toute précision.

In [78]:
Random.init 42

- : unit = ()


In [79]:
list_shuffle [1; 2; 3; 4; 5; 6; 7; 8; 9]

- : int list = [3; 5; 4; 9; 8; 6; 2; 7; 1]


### `play`
La fonction
```ocaml
val play : int -> string -> unit = <fun>
```
contrôle le déroulement d'une partie à partir d'une chaîne de caractères `'B'` et `'H'` et du plus haut chiffre d'un domino. Elle mélange ces dominos, les distribue aux joueurs, affiche la configuration initiale, déroule le jeu et affiche à la fin quel joueur a gagné ou, le cas échéant, `"Match nul!"`.

**Remarque.** La fonction renvoyant `()`, son résultat n'est pas testé dans `test.ml`. Il vous appartient de vérifier que vous obtenez les mêmes effets de bord que dans les deux exemples de parties ci-dessous.

In [80]:
read := (let x = Stream.of_list ["00"; "22"; "30"; ">"; "40"; "10"; ">"] in function () -> Stream.next x);
Random.init 42;
play 4 "HB"

Joueur 1 (humain):	0-0 2-1 3-0 4-1 2-2 4-0 4-2
Joueur 2 (bot)   :	4-4 3-2 4-3 3-3 2-0 3-1 1-1
Joueur 1 (humain):
	main : 0-0 2-1 3-0 4-1 2-2 4-0 4-2
	Quel domino voulez-vous poser ? 
	chaîne :             0-0
Joueur 2 (bot)   :
	main : 4-4 3-2 4-3 3-3 2-0 3-1 1-1
	coup forcé :         2-0
	chaîne :             2-0 0-0
Joueur 1 (humain):
	main : 2-1 3-0 4-1 2-2 4-0 4-2
	Quel domino voulez-vous poser ? 
	chaîne :             2-2 2-0 0-0
Joueur 2 (bot)   :
	main : 4-4 3-2 4-3 3-3 3-1 1-1
	coup forcé :         3-2
	chaîne :             3-2 2-2 2-0 0-0
Joueur 1 (humain):
	main : 2-1 3-0 4-1 4-0 4-2
	Quel domino voulez-vous poser ? 
	À quel bout ? 
	chaîne :             3-2 2-2 2-0 0-0 0-3
Joueur 2 (bot)   :
	main : 4-4 4-3 3-3 3-1 1-1
	à placer :           3-3
	chaîne :             3-3 3-2 2-2 2-0 0-0 0-3
Joueur 1 (humain):
	main : 2-1 4-1 4-0 4-2
	Aucun coup possible!
	chaîne :             3-3 3-2 2-2 2-0 0-0 0-3
Joueur 2 (bot)   :
	main : 4-4 4-3 3-1 1-1
	à placer :           4-3
	chaîne 

- : unit = ()


In [81]:
Random.init 42;
play 6 "BBB"

Joueur 1 (bot)   :	5-4 6-3 5-3 0-0 6-2 6-4
Joueur 2 (bot)   :	5-0 4-3 4-1 3-3 3-0 5-2
Joueur 3 (bot)   :	4-0 6-1 1-0 2-0 5-1 5-5
Joueur 1 (bot)   :
	main : 5-4 6-3 5-3 0-0 6-2 6-4
	à placer :           5-3
	chaîne :             5-3
Joueur 2 (bot)   :
	main : 5-0 4-3 4-1 3-3 3-0 5-2
	à placer :           5-2
	chaîne :             2-5 5-3
Joueur 3 (bot)   :
	main : 4-0 6-1 1-0 2-0 5-1 5-5
	coup forcé :         2-0
	chaîne :             0-2 2-5 5-3
Joueur 1 (bot)   :
	main : 5-4 6-3 0-0 6-2 6-4
	à placer :           6-3
	chaîne :             0-2 2-5 5-3 3-6
Joueur 2 (bot)   :
	main : 5-0 4-3 4-1 3-3 3-0
	à placer :           3-0
	chaîne :             3-0 0-2 2-5 5-3 3-6
Joueur 3 (bot)   :
	main : 4-0 6-1 1-0 5-1 5-5
	coup forcé :         6-1
	chaîne :             3-0 0-2 2-5 5-3 3-6 6-1
Joueur 1 (bot)   :
	main : 5-4 0-0 6-2 6-4
	Aucun coup possible!
	chaîne :             3-0 0-2 2-5 5-3 3-6 6-1
Joueur 2 (bot)   :
	main : 5-0 4-3 4-1 3-3
	à placer :           4-3
	chaîne :             4-3

- : unit = ()
